### Handling Captchas in a demo site

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.google.com/recaptcha/api2/demo")

# Switch to the reCAPTCHA iframe
iframe = driver.find_element(By.XPATH, "//iframe[contains(@src, 'recaptcha')]")
driver.switch_to.frame(iframe)

# Click the checkbox
checkbox = driver.find_element(By.CLASS_NAME, "recaptcha-checkbox-border")
checkbox.click()

# Switch back to main content
driver.switch_to.default_content()

print("Clicked the reCAPTCHA checkbox.")
time.sleep(10)  # Wait to see it before quitting
driver.quit()


Clicked the reCAPTCHA checkbox. You may still need to solve images manually.


### Handling Captchas using a 3rd party (2Captcha)

Note that this would require paid features and hence will be considered out of scope - only demonstrated for additional knowledge

In [ ]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

API_KEY = "ADD API KEY"  # Get from https://2captcha.com/

# 1. Open browser and go to test page
driver = webdriver.Chrome()
driver.get("https://www.google.com/recaptcha/api2/demo")

# 2. Get sitekey from the page (visible in HTML)
site_key = driver.find_element(By.CLASS_NAME, "g-recaptcha").get_attribute("data-sitekey")
print("Site Key:", site_key)

# 3. Send sitekey + page URL to 2Captcha
url = "https://www.google.com/recaptcha/api2/demo"
resp = requests.post(
    "http://2captcha.com/in.php",
    data={
        "key": API_KEY,
        "method": "userrecaptcha",
        "googlekey": site_key,
        "pageurl": url,
        "json": 1
    }
)
request_id = resp.json()["request"]
print("Captcha Request ID:", request_id)

# 4. Poll for the result
recaptcha_answer = None
for _ in range(20):
    time.sleep(5)  # wait between polls
    result = requests.get(
        f"http://2captcha.com/res.php?key={API_KEY}&action=get&id={request_id}&json=1"
    )
    if result.json()["status"] == 1:
        recaptcha_answer = result.json()["request"]
        break

if recaptcha_answer:
    print("CAPTCHA Solved!")
    
    # 5. Inject token into the page
    driver.execute_script(
        f'document.getElementById("g-recaptcha-response").innerHTML="{recaptcha_answer}";'
    )
    time.sleep(1)

    # 6. Submit the form
    driver.find_element(By.ID, "recaptcha-demo-submit").click()

    print("Form submitted after CAPTCHA solve.")
else:
    print("Failed to solve CAPTCHA.")

time.sleep(5)
driver.quit()


Site Key: 6Le-wvkSAAAAAPBMRTvw0Q4Muexq9bi0DJwx_mJ-
Captcha Request ID: ERROR_ZERO_BALANCE
Failed to solve CAPTCHA.
